<a href="https://colab.research.google.com/github/MoisesLeggis/POO/blob/main/Proyecto_Final_LeggisMoises_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install streamlit pyngrok
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦
changed 22 packages in 2s
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧

In [55]:
%%writefile app.py
import streamlit as st
from datetime import date
import pickle
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
import requests
from PIL import Image
from io import BytesIO
import sys

#LEGGIS GOMEZ MOISES ALEJANDRO CODIGO ---> 218423359
# --- Clases ---

class Birthday:
    """
    Clase que representa un cumpleaños con información personal y métodos relacionados.
    """

    def __init__(self, name: str, birth_date: date, email: str, message_id: int = None):
        """
        Inicializa una instancia de Birthday.

        Args:
            name (str): Nombre de la persona
            birth_date (date): Fecha de cumpleaños
            email (str): Email de la persona
            message_id (int, optional): ID del mensaje personalizado. Defaults to None.
        """
        self.name = name
        self.birth_date = birth_date
        self.email = email
        self.message_id = message_id

    def days_remaining(self) -> int:
        """
        Calcula los días restantes hasta el próximo cumpleaños.

        Returns:
            int: Días restantes hasta el próximo cumpleaños
        """
        today = date.today()
        next_birthday = date(today.year, self.birth_date.month, self.birth_date.day)
        if next_birthday < today:
            next_birthday = date(today.year + 1, self.birth_date.month, self.birth_date.day)
        return (next_birthday - today).days

    def is_today(self) -> bool:
        """
        Verifica si hoy es el cumpleaños.

        Returns:
            bool: True si hoy es el cumpleaños, False en caso contrario
        """
        today = date.today()
        return today.month == self.birth_date.month and today.day == self.birth_date.day


class BirthdayManager:
    """
    Clase principal que gestiona todos los cumpleaños y mensajes.
    Maneja almacenamiento, recuperación y operaciones relacionadas.
    """

    def __init__(self):
        """Inicializa el gestor de cumpleaños cargando datos desde archivo."""
        self.birthdays = []
        self.messages = [
            "¡Feliz cumpleaños! Que tengas un día maravilloso.",
            "Feliz cumpleaños. ¡Que todos tus deseos se hagan realidad!",
            "Feliz cumpleaños. ¡Disfruta tu día especial!"
        ]
        self.load_from_file()

    def add_birthday(self, name: str, birth_date: date, email: str, message_id: int = None):
        """
        Añade un nuevo cumpleaños a la lista y guarda en archivo.

        Args:
            name (str): Nombre de la persona
            birth_date (date): Fecha de cumpleaños
            email (str): Email de la persona
            message_id (int, optional): ID del mensaje personalizado. Defaults to None.
        """
        self.birthdays.append(Birthday(name, birth_date, email, message_id))
        self.save_to_file()

    def remove_birthday(self, name: str):
        """
        Elimina un cumpleaños por nombre y guarda en archivo.

        Args:
            name (str): Nombre de la persona a eliminar
        """
        self.birthdays = [b for b in self.birthdays if b.name != name]
        self.save_to_file()

    def get_upcoming(self, days: int = 30):
        """
        Obtiene los cumpleaños próximos dentro de un rango de días.

        Args:
            days (int, optional): Rango de días para buscar. Defaults to 30.

        Returns:
            list: Lista de cumpleaños ordenados por días restantes
        """
        upcoming = [b for b in self.birthdays if b.days_remaining() <= days]
        return sorted(upcoming, key=lambda x: x.days_remaining())

    def add_message(self, message: str):
        """
        Añade un nuevo mensaje personalizado y guarda en archivo.

        Args:
            message (str): Texto del mensaje a añadir
        """
        self.messages.append(message)
        self.save_to_file()

    def get_message(self, message_id: int = None):
        """
        Obtiene un mensaje por ID o uno aleatorio si no se especifica.

        Args:
            message_id (int, optional): ID del mensaje a recuperar. Defaults to None.

        Returns:
            str: El mensaje seleccionado
        """
        if not self.messages:
            return "¡Feliz cumpleaños!"
        if message_id is not None and 0 <= message_id < len(self.messages):
            return self.messages[message_id]
        import random
        return random.choice(self.messages)

    def save_to_file(self, filename: str = 'birthdays.pkl'):
        """
        Guarda los datos de cumpleaños y mensajes en un archivo pickle.

        Args:
            filename (str, optional): Nombre del archivo. Defaults to 'birthdays.pkl'.
        """
        with open(filename, 'wb') as f:
            pickle.dump({
                'birthdays': [(b.name, b.birth_date, b.email, b.message_id) for b in self.birthdays],
                'messages': self.messages
            }, f)

    def load_from_file(self, filename: str = 'birthdays.pkl'):
        """
        Carga los datos de cumpleaños y mensajes desde un archivo pickle.

        Args:
            filename (str, optional): Nombre del archivo. Defaults to 'birthdays.pkl'.
        """
        try:
            with open(filename, 'rb') as f:
                data = pickle.load(f)
                self.birthdays = [
                    Birthday(name, birth_date, email, message_id)
                    for name, birth_date, email, message_id in data['birthdays']
                ]
                self.messages = data['messages']
        except FileNotFoundError:
            pass  # Archivo no existe aún, se inicia con valores por defecto

    def send_automatic_greetings(self, email_sender, image_url: str = None):
        """
        Envía felicitaciones automáticas a todos los cumpleañeros de hoy.

        Args:
            email_sender: Instancia de EmailSender para enviar los correos
            image_url (str, optional): URL de imagen para incluir. Defaults to None.
        """
        today_bdays = [b for b in self.birthdays if b.is_today()]
        for bday in today_bdays:
            message = self.get_message(bday.message_id)
            error = email_sender.send_email_with_image(
                bday.email,
                f"Feliz Cumpleaños {bday.name}!",
                message,
                image_url
            )
            if error:
                print(f"Error enviando email a {bday.email}: {error}")
            else:
                print(f"Email enviado exitosamente a {bday.email}")


class EmailSender:
    """
    Clase para manejar el envío de emails con soporte para imágenes adjuntas.
    """

    def __init__(self, smtp_server: str, smtp_port: int, sender_email: str, sender_pass: str):
        """
        Inicializa el EmailSender con credenciales SMTP.

        Args:
            smtp_server (str): Servidor SMTP
            smtp_port (int): Puerto SMTP
            sender_email (str): Email del remitente
            sender_pass (str): Contraseña del remitente
        """
        self.smtp_server = smtp_server
        self.smtp_port = smtp_port
        self.sender_email = sender_email
        self.sender_pass = sender_pass

    def send_email_with_image(self, to_email: str, subject: str, text: str, image_url: str = None):
        """
        Envía un email con texto opcionalmente incluyendo una imagen desde URL.

        Args:
            to_email (str): Email del destinatario
            subject (str): Asunto del email
            text (str): Cuerpo del mensaje
            image_url (str, optional): URL de imagen para incluir. Defaults to None.

        Returns:
            str: Mensaje de error si falla, None si tiene éxito
        """
        msg = MIMEMultipart()
        msg['Subject'] = subject
        msg['From'] = self.sender_email
        msg['To'] = to_email

        msg.attach(MIMEText(text, 'plain'))

        if image_url:
            try:
                response = requests.get(image_url)
                img = MIMEImage(response.content)
                img.add_header('Content-ID', '<birthday_image>')
                msg.attach(img)
                text += f"\n\n<img src='cid:birthday_image'>"
            except Exception as e:
                st.warning(f"No se pudo cargar la imagen: {str(e)}")

        try:
            with smtplib.SMTP(self.smtp_server, self.smtp_port) as server:
                server.starttls()
                server.login(self.sender_email, self.sender_pass)
                server.send_message(msg)
            return None
        except Exception as e:
            return str(e)


# --- Interfaz Streamlit ---
def main():
    """
    Función principal que configura la interfaz de usuario con Streamlit.
    """
    st.set_page_config(page_title="🎂 Gestor de Cumpleaños", layout="wide")
    manager = BirthdayManager()

    # Sidebar - Configuración SMTP
    with st.sidebar:
        st.header("⚙️ Configuración de Email")
        smtp_server = st.text_input("Servidor SMTP", "smtp.gmail.com")
        smtp_port = st.number_input("Puerto SMTP", 587)
        sender_email = st.text_input("Tu Email")
        sender_pass = st.text_input("Tu Contraseña", type="password")
        image_url = st.text_input("URL de Imagen", "https://pbs.twimg.com/media/EBN8aNCWsAEvAKK.jpg")

        auto_send = st.checkbox("Habilitar envío automático")

        if st.button("Guardar Configuración"):
            try:
                email_sender = EmailSender(smtp_server, smtp_port, sender_email, sender_pass)
                st.session_state.email_sender = email_sender
                st.session_state.image_url = image_url
                st.session_state.auto_send = auto_send
                st.success("¡Configuración guardada!")

                # Envío automático al guardar la configuración (si está habilitado)
                if auto_send:
                    with st.spinner("Enviando felicitaciones automáticas..."):
                        manager.send_automatic_greetings(email_sender, image_url)
                    st.success("¡Felicitaciones automáticas enviadas!")
            except Exception as e:
                st.error(f"Error: {str(e)}")

    # Pestañas principales
    tab1, tab2, tab3, tab4 = st.tabs(["🏠 Inicio", "➕ Agregar", "💬 Mensajes", "✉️ Envío"])

    with tab1:
        st.header("Próximos Cumpleaños")
        upcoming = manager.get_upcoming(30)
        if not upcoming:
            st.info("No hay cumpleaños próximos en los próximos 30 días")
        else:
            for idx, bday in enumerate(upcoming):
                with st.expander(f"{bday.name} - {bday.birth_date.strftime('%d/%m')}"):
                    st.write(f"📧 {bday.email}")
                    st.write(f"⏳ Faltan {bday.days_remaining()} días")
                    if st.button("Eliminar", key=f"del_{idx}_{bday.name}"):
                        manager.remove_birthday(bday.name)
                        st.rerun()

    with tab2:
        st.header("Agregar Cumpleaños")
        with st.form("add_form"):
            name = st.text_input("Nombre")
            birth_date = st.date_input("Fecha")
            email = st.text_input("Email")
            message_id = st.selectbox("Mensaje predeterminado",
                                    options=[(i, msg) for i, msg in enumerate(manager.messages)],
                                    format_func=lambda x: x[1],
                                    index=None)

            if st.form_submit_button("Guardar"):
                if name and email:
                    manager.add_birthday(name, birth_date, email, message_id[0] if message_id else None)
                    st.success("¡Guardado!")
                    st.rerun()

    with tab3:
        st.header("Mensajes Personalizados")
        st.subheader("Tus Mensajes")
        for i, msg in enumerate(manager.messages):
            cols = st.columns([4, 1])
            cols[0].write(msg)
            if cols[1].button("❌", key=f"del_msg_{i}"):
                manager.messages.pop(i)
                manager.save_to_file()
                st.rerun()

        st.subheader("Nuevo Mensaje")
        new_msg = st.text_area("Mensaje", "Feliz cumpleaños\nmi princesa le\nqueremos mucha!")
        if st.button("Añadir"):
            if new_msg:
                manager.add_message(new_msg)
                st.rerun()

    with tab4:
        st.header("Enviar Felicitaciones")
        if 'email_sender' not in st.session_state:
            st.warning("Configura el email primero")
        else:
            today_bdays = [b for b in manager.birthdays if b.is_today()]
            if not today_bdays:
                st.info("No hay cumpleaños hoy")
            else:
                for idx, bday in enumerate(today_bdays):
                    with st.container(border=True):
                        st.subheader(bday.name)
                        msg = st.selectbox("Mensaje", manager.messages, key=f"msg_{idx}_{bday.name}")
                        if st.button("Enviar", key=f"send_{idx}_{bday.name}"):
                            with st.spinner("Enviando..."):
                                error = st.session_state.email_sender.send_email_with_image(
                                    bday.email,
                                    f"Feliz Cumpleaños {bday.name}!",
                                    msg,
                                    st.session_state.image_url
                                )
                            if error:
                                st.error(f"Error: {error}")
                            else:
                                st.success("¡Enviado!")

if __name__ == "__main__":
    # Verificación automática al inicio
    manager = BirthdayManager()
    if any(b.is_today() for b in manager.birthdays):
        print("Detectando cumpleaños hoy...")

        # Intenta cargar configuración previa para envío automático
        try:
            with open('email_config.pkl', 'rb') as f:
                config = pickle.load(f)
                email_sender = EmailSender(
                    config['smtp_server'],
                    config['smtp_port'],
                    config['sender_email'],
                    config['sender_pass']
                )
                image_url = config['image_url']
                auto_send = config['auto_send']

                if auto_send:
                    print("Enviando felicitaciones automáticas...")
                    manager.send_automatic_greetings(email_sender, image_url)
                    print("Felicitaciones enviadas!")
        except FileNotFoundError:
            print("Configuración de email no encontrada. Ejecuta la app para configurar.")

    # Ejecutar la aplicación
    main()

Overwriting app.py


In [56]:
# Usado para borrar el contenido de los cumpleanos
!rm -f /content/birthdays.pkl

In [57]:
#Aqui para cerrar todos los tuneles creados en ngrok  ( No sera necesario utilizarlo)
from pyngrok import conf, ngrok

# Cargar configuración actual
config = conf.get_default()

# Cerrar todos los túneles abiertos
for tunnel in ngrok.get_tunnels():
    ngrok.disconnect(tunnel.public_url)


In [58]:
# 1. Instalar dependencias (si no están instaladas)
!pip install -q pyngrok streamlit

# 2. Escribir el archivo app.py (usa %%writefile antes si lo tienes en una celda separada)
# Ejemplo mínimo para pruebas:
with open("app.py", "w") as f:
    f.write("import streamlit as st\nst.title('Hola desde Streamlit en Colab')")

# 3. Configurar ngrok
from pyngrok import ngrok
ngrok.set_auth_token("2xHbl7P5YPhHKTZZHJz1UxoFbfo_3dQ4ypMrdy124AHX3MxjP")  # Verifica que el token esté vigente

# 4. Iniciar Streamlit en segundo plano
import subprocess
import threading
import time

def run_streamlit():
    subprocess.run([
        "streamlit", "run", "app.py",  # No uses /content/app.py directamente, usa solo "app.py"
        "--server.port", "8501",
        "--server.headless", "true",
        "--server.enableCORS", "false",
        "--server.enableXsrfProtection", "false"
    ])

thread = threading.Thread(target=run_streamlit)
thread.start()

# 5. Esperar a que se levante el servidor
time.sleep(5)

# 6. Crear túnel con ngrok
public_url = ngrok.connect(8501, bind_tls=True).public_url
print(f"✅ Tu aplicación está disponible en:\n{public_url}")

✅ Tu aplicación está disponible en:
https://f302-34-48-135-74.ngrok-free.app
